<a href="https://colab.research.google.com/github/Rodolfoemmel/projeto-viagem/blob/main/Pacotes_de_viagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))



Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição foram até o dia 11 de maio de 2025.


In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição foram até o dia 11 de maio de 2025.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini datas', 'Próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: eucapacito.com.br, starten.tech, convergenciadigital.com.br, tecmundo.com.br, youtube.com, alura.com.br



In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para pesquisa as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Seu tema pacote de viagens com os preços mais baixos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            o seu objetivo é detalhar os preços dos pacotes em uma lista apresentavel escrita de forma formal para pessoas de idade entre 25-40 anos.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema buscador de pacote de viagens com 4 Agentes de IA")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o nome do local que você deseja viajar:")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o local")
else:
    print(f"Maravilha! Vamos então criar uma busca completa para a sua viagem a {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

Iniciando o Sistema buscador de pacote de viagens com 4 Agentes de IA
❓ Por favor, digite o nome do local que você deseja viajar:brasilia
Maravilha! Vamos então criar uma busca completa para a sua viagem a brasilia

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os pacotes de viagens com os preços mais baixos para Brasília, focando em lançamentos recentes (dentro do último mês), vou realizar algumas pesquisas no Google.
> 
> 
> Com base nas pesquisas, aqui estão algumas opções e dicas para encontrar pacotes de viagens baratos para Brasília em maio de 2025:
> 
> **Opções de Pacotes e Agências:**
> 
> *   **KAYAK:** Oferece pacotes de viagem para Brasília a partir de R$ 2.246. Eles também mostram opções de "férias de baixo custo" a partir de R$ 944 para 3 dias e duas pessoas.
> *   **Viajanet:** Menciona pacotes de viagem baratos até R$ 1.000, com saídas de Brasília para outros destinos.
> *   **Decolar:** Permite comprar pacotes para Brasília em até 12x sem juros.
> *   **CVC:** Tem promoções de viagens para destinos nacionais, incluindo pacotes com voo + hotel e condições de pagamento facilitadas.
> *   **Hurb:** Oferece um pacote de voo + hotel para Brasília em 2025.
> *   **Latam Travel (em parceria com Decolar):** Menciona descontos em pacotes (voo + hotel) e opções de parcelamento.
> *   **Melhores Destinos:** Avalia pacotes de várias agências (Viajanet, Zarpo, CVC, Decolar) e seleciona as melhores ofertas.
> 
> **Dicas para Encontrar Pacotes Mais Baratos:**
> 
> *   **Comparar Preços:** Utilize sites comparadores como KAYAK e Melhores Destinos para avaliar as opções de diferentes agências.
> *   **Flexibilidade de Datas:** Seja flexível com as datas da viagem, pois os preços podem variar dependendo do dia da semana e da época do ano.
> *   **Detalhes do Pacote:** Verifique o que está incluído no pacote (voo, hotel, traslados, etc.) e a localização do hotel.
> *   **Comprar com Antecedência:** Em geral, reservar com antecedência pode garantir melhores preços.
> *   **Promoções:** Fique atento às promoções e ofertas especiais das agências de viagem.
> *   **Sites de Confiança:** Compre em sites de agências de viagem conhecidas e confiáveis.
> 
> **Ofertas Específicas Mencionadas:**
> 
> *   **Voos de Brasília:** A Skyscanner menciona passagens aéreas baratas saindo de Brasília a partir de R$ 219. A GOL também oferece voos de Brasília a partir de R$ 262,69.
> *   **Pacote CVC:** Pacotes com voo de volta com bagagem inclusa e voo direto, com saída de São Paulo.
> 
> **Observações:**
> 
> *   As ofertas e preços podem variar dependendo da disponibilidade e das datas da viagem.
> *   É importante verificar os termos e condições de cada pacote antes de efetuar a compra.
> *   Algumas ofertas podem ter restrições de datas ou exigir um número mínimo de passageiros.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nas informações fornecidas, podemos criar um plano de conteúdo para um post sobre pacotes de viagem baratos para Brasília, focando nos seguintes pontos:
> 
> **TEMA PRINCIPAL: Encontre Pacotes de Viagem Acessíveis para Brasília em Maio de 2025**
> 
> **Pontos Relevantes a Abordar:**
> 
> 1.  **Panorama Geral das Opções:**
>     *   Mencionar as principais agências que oferecem pacotes para Brasília (KAYAK, Viajanet, Decolar, CVC, Hurb, Latam Travel).
>     *   Destacar que KAYAK oferece pacotes a partir de R$2.246 e opções de "férias de baixo custo" a partir de R$944 (3 dias, 2 pessoas).
>     *   Referenciar que a Viajanet menciona pacotes baratos até R$1.000 com saídas de Brasília para outros destinos.
>     *   Ressaltar a possibilidade de parcelamento em até 12x sem juros na Decolar.
> 
> 2.  **Dicas Essenciais para Economizar:**
>     *   **Comparação de Preços:** Enfatizar a importância de usar comparadores como KAYAK e Melhores Destinos.
>     *   **Flexibilidade de Datas:** Explicar como datas flexíveis podem impactar os preços.
>     *   **Análise Detalhada:** Aconselhar a verificação dos itens inclusos no pacote e a localização do hotel.
>     *   **Compra Antecipada:** Reforçar que reservar com antecedência geralmente garante melhores preços.
>     *   **Atenção às Promoções:** Incentivar os leitores a ficarem de olho em ofertas especiais.
>     *   **Segurança:** Recomendar a compra em sites de agências confiáveis.
> 
> 3.  **Ofertas Específicas e Voos:**
>     *   Citar ofertas de voos a partir de R$219 (Skyscanner) e R$262,69 (GOL) saindo de Brasília.
>     *   Mencionar pacotes da CVC com voo de volta, bagagem inclusa e voo direto, partindo de São Paulo.
> 
> 4.  **Alertas e Observações Importantes:**
>     *   Frisar que os preços e ofertas podem variar.
>     *   Relembrar a importância de verificar os termos e condições de cada pacote.
>     *   Alertar sobre possíveis restrições de datas ou exigência de número mínimo de passageiros.
> 
> **Plano Detalhado para o Post:**
> 
> *   **Título Atraente:** Exemplo: "Pacotes Imperdíveis para Brasília em Maio de 2025: Economize e Viaje!"
> *   **Introdução:**
>     *   Breve descrição sobre Brasília como destino turístico.
>     *   Afirmar o objetivo do post: ajudar a encontrar pacotes de viagem acessíveis.
> *   **Corpo do Post:**
>     *   Dividir o conteúdo em seções, cada uma focada em um dos pontos relevantes acima.
>     *   Utilizar títulos e subtítulos para facilitar a leitura.
>     *   Incluir exemplos de preços e ofertas encontrados.
>     *   Inserir links para os sites das agências mencionadas.
> *   **Destaque Visual:**
>     *   Adicionar fotos de Brasília para atrair o público.
>     *   Criar gráficos comparando preços de diferentes agências (se possível).
> *   **Chamada para Ação (CTA):**
>     *   Incentivar os leitores a pesquisarem os pacotes nos sites das agências.
>     *   Convidar os leitores a compartilharem o post com amigos.
>     *   Solicitar que deixem comentários sobre suas experiências e dicas.
> 
> **Tema Mais Relevante:**
> 
> O tema mais relevante, considerando o objetivo de auxiliar o público a encontrar opções acessíveis, é: **"Como Encontrar Pacotes de Viagem Baratos para Brasília em Maio de 2025: Guia Completo"**.
> 
> **Justificativa:** Este título é direto, informativo e promete um guia completo, o que atrai mais leitores interessados em economizar em suas viagens. Ele engloba todos os pontos relevantes mencionados e permite uma abordagem detalhada e prática.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## ✈️ Descubra Brasília sem Gastar uma Fortuna em Maio de 2025! 🏛️
> 
> Brasília, a capital do Brasil, te espera com sua arquitetura icônica e cultura vibrante! Mas, calma, você não precisa gastar rios de dinheiro para conhecer essa cidade incrível. 😉
> 
> Preparamos um guia completo para você encontrar **pacotes de viagem acessíveis para Brasília em maio de 2025**. Chega de adiar o sonho de conhecer a capital federal! 🚀
> 
> **🔍 Onde Encontrar os Melhores Pacotes?**
> 
> *   **KAYAK:** Fique de olho! Eles têm pacotes a partir de R$2.246 e opções "econômicas" a partir de R$944 (para 2 pessoas, 3 dias). 💰
> *   **Viajanet:** Sabia que você pode encontrar pacotes de Brasília para outros destinos por menos de R$1.000? 🤯 Vale a pena conferir!
> *   **Decolar:** Parcelamento em até 12x sem juros? Sim, é na Decolar! 💳 Uma ótima opção para quem quer planejar a viagem com calma.
> *   **CVC, Hurb e Latam Travel:** Agências tradicionais que também oferecem ótimas opções de pacotes. Não deixe de comparar! 😉
> 
> **💰 Dicas de Ouro para Economizar:**
> 
> *   **Compare, compare, compare!** Use sites como KAYAK e Melhores Destinos para ter uma visão geral dos preços.
> *   **Flexibilidade é a chave!** Datas flexíveis podem significar uma GRANDE economia. 😉
> *   **Analise tudo!** O que está incluso no pacote? Onde fica o hotel? Pesquise antes de fechar!
> *   **Antecipe-se!** Reservar com antecedência quase sempre garante os melhores preços. 🗓️
> *   **De olho nas promoções!** As agências sempre lançam ofertas especiais. Fique ligado! 👀
> *   **Segurança em primeiro lugar!** Compre em sites de agências confiáveis. ✅
> 
> **✨ Ofertas Especiais (Fique Atento!):**
> 
> *   **Voos:** Skyscanner (a partir de R$219) e GOL (a partir de R$262,69) saindo de Brasília. ✈️
> *   **CVC:** Pacotes com voo de volta, bagagem inclusa e voo direto (saindo de São Paulo). 🤩
> 
> **⚠️ Atenção!**
> 
> *   Preços e ofertas podem variar a qualquer momento.
> *   Verifique os termos e condições de cada pacote antes de comprar.
> *   Alguns pacotes podem ter restrições de datas ou exigir um número mínimo de passageiros.
> 
> **👉 E aí, pronto para conhecer Brasília sem estourar o orçamento?**
> 
> Clique nos links das agências e comece a planejar sua viagem agora mesmo! 😉
> 
> **✅ Marque seus amigos que também sonham em conhecer Brasília!**
> 
> **💬 Compartilhe suas dicas e experiências nos comentários!**
> 
> #Brasília #PacotesDeViagem #ViagemBarata #Maio2025 #DicasDeViagem #EconomizeEViaje #TurismoBrasil #ViagemDosSonhos
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> ## ✈️ Partiu Brasília Sem Gastar Uma Fortuna? (Sim, é Possível!) 🏛️
> 
> Quem disse que conhecer a capital do Brasil precisa pesar no bolso? 🤔 Brasília te espera com sua arquitetura única e uma vibe cultural incrível, e a gente te mostra como aproveitar ao máximo SEM estourar o orçamento! 😉
> 
> Está sonhando em conhecer a cidade? Preparamos um guia COMPLETO com dicas e macetes para você encontrar os melhores pacotes de viagem e voos promocionais! 🚀
> 
> **💰 O Segredo é Comparar (e Ser Esperto!):**
> 
> *   Use comparadores de preço como Kayak e Melhores Destinos para garimpar as melhores ofertas. 🔍
> *   Flexibilidade é TUDO! Se puder viajar em datas alternativas, a economia pode ser ENORME. 🗓️
> *   Fique de olho nas promoções relâmpago das companhias aéreas e agências de viagem. ⚡
> *   Considere viajar na baixa temporada (evite feriados e meses de férias escolares). 😉
> *   Pesquise MUITO antes de fechar qualquer pacote! Veja o que está incluso, a localização do hotel e as avaliações de outros viajantes. 👀
> 
> **✨ Dicas Extras Que Valem Ouro:**
> 
> *   Explore a cidade de ônibus ou aplicativos de transporte – é mais barato que alugar um carro! 🚌
> *   Aproveite os museus e parques com entrada gratuita. Brasília tem muita coisa legal para ver sem gastar nada! 🤩
> *   Experimente a culinária local nos restaurantes e mercados populares – você vai se surpreender com os sabores e os preços! 😋
> 
> **👉 Preparado(a) para desbravar Brasília sem gastar uma fortuna?**
> 
> Marca nos comentários aquele amigo(a) que precisa saber dessas dicas e bora planejar essa trip! 👇
> 
> #Brasília #ViagemEconômica #DicasDeViagem #TurismoBrasil #PartiuBrasília
> 


--------------------------------------------------------------
